In [ ]:
# **************** UNCOMMENT AND RUN THIS CELL IF YOU NEED TO INSTALL/ UPGRADE TSAI ****************
stable = True # Set to True for latest pip version or False for main branch in GitHub
!pip install {"tsai -U" if stable else "git+https://github.com/timeseriesAI/tsai.git"} >> /dev/null
from tsai.all import *
computer_setup()

os              : Linux-6.1.58+-x86_64-with-glibc2.35
python          : 3.10.12
tsai            : 0.3.9
fastai          : 2.7.14
fastcore        : 1.5.29
torch           : 2.2.1+cu121
device          : cpu
cpu cores       : 1
threads per cpu : 2
RAM             : 12.67 GB
GPU memory      : N/A


In [ ]:
data = pd.read_csv('/content/gdrive/MyDrive/UND/Fires/FireBurntArea_dataset_0Days.csv',header=0)
data

,LC_Type1,LC_Type2,LC_Type3,LC_Type4,LC_Type5,avgLongitude,avgLatitude,Elevation,Slope,Aspect,...,runoff_sum,total_evaporation_sum,snowfall_sum,snowmelt_sum,snow_depth,snow_density,snow_cover,dewpoint_temperature_2m,temperature_2m,totalArea_km2
0,8,8,4,4,4,-162.81695,62.953425,76,0.127839,275.667218,...,0.000868,-0.000535,0.000005,0.000000e+00,1.690267e-01,198.237289,100.000000,265.185745,269.474932,0.979713
1,8,8,4,4,4,-147.15870,64.785600,151,0.267640,346.948073,...,0.000046,-0.002125,0.000000,1.543760e-05,-7.345365e-24,154.937485,0.102783,276.366584,281.858940,0.295989
2,8,8,4,1,1,-148.53950,64.364500,174,0.289804,354.668166,...,0.000249,-0.003539,0.000000,0.000000e+00,-7.345365e-24,100.002263,0.000000,281.229359,291.656452,0.250000
3,7,7,2,1,5,-146.50850,63.993100,580,0.545915,285.295886,...,0.001994,-0.000478,0.000000,3.360212e-06,-7.345365e-24,161.781235,0.082926,270.544548,274.700846,0.250000
4,7,7,2,1,5,-146.35220,64.024000,453,0.447008,299.675079,...,0.000587,-0.000995,0.000000,4.351754e-03,2.640788e-02,267.080388,26.630371,273.304184,282.790248,0.250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2065,8,8,4,1,1,-145.93715,64.643550,313,0.400006,334.382444,...,0.000534,-0.002531,0.000000,7.450581e-09,-7.345365e-24,105.813136,0.000000,280.222176,287.991054,0.830562
2066,15,15,9,7,10,-161.88630,55.418800,2186,0.025017,270.000000,...,0.006850,-0.000637,0.000000,6.294075e-03,1.344824e+01,180.705714,40.343750,281.538507,282.846760,0.250000
2067,15,15,9,7,10,-161.89350,55.417900,2479,0.025017,270.000000,...,0.006829,-0.001089,0.000000,6.279286e-03,1.344824e+01,180.694321,40.343750,278.835497,280.776448,0.250000
2068,15,15,9,7,10,-161.89090,55.422600,2100,0.025017,270.000000,...,0.004876,-0.000546,0.000000,3.418032e-03,1.344824e+01,194.436834,40.363200,275.874074,277.500503,0.250000


In [ ]:
import numpy as np
from scipy.stats import rankdata, norm


def xicor(x, y, ties="auto"):
    x = np.asarray(x).flatten()
    y = np.asarray(y).flatten()
    n = len(y)

    if len(x) != n:
        raise IndexError(
            f"x, y length mismatch: {len(x)}, {len(y)}"
        )

    if ties == "auto":
        ties = len(np.unique(y)) < n
    elif not isinstance(ties, bool):
        raise ValueError(
            f"expected ties either \"auto\" or boolean, "
            f"got {ties} ({type(ties)}) instead"
        )

    y = y[np.argsort(x)]
    r = rankdata(y, method="ordinal")
    nominator = np.sum(np.abs(np.diff(r)))

    if ties:
        l = rankdata(y, method="max")
        denominator = 2 * np.sum(l * (n - l))
        nominator *= n
    else:
        denominator = np.power(n, 2) - 1
        nominator *= 3

    statistic = 1 - nominator / denominator  # upper bound is (n - 2) / (n + 1)
    p_value = norm.sf(statistic, scale=2 / 5 / np.sqrt(n))

    return statistic, p_value

In [ ]:
import pandas as pd
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
from scipy.stats import spearmanr, pearsonr
from sklearn.feature_selection import mutual_info_regression,mutual_info_classif
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

# Load your dataset
data = pd.read_csv('/content/gdrive/MyDrive/UND/Fires/FireDetection_dataset_0Days.csv',header=0)
labels = ['LC_Type1', 'avgLatitude', 'Slope', 'Aspect', 'snowfall_sum'] + ['isFire']
data = data[labels]

# Separate the independent variables and the target variable
X = data.drop('isFire', axis=1)
y = data['isFire']

# Add a constant term for the intercept to the independent variables
X_const = add_constant(X)

# Initialize DataFrame to store the analysis results
analysis_df = pd.DataFrame()
analysis_df['Feature'] = X.columns

# Compute VIF
analysis_df['VIF'] = [variance_inflation_factor(X_const.values, i+1) for i in range(len(X.columns))]  # i+1 to skip constant
# analysis_df['Tolerance'] = 1 / analysis_df['VIF']

# # Compute Mutual Information
# analysis_df['Mutual Information'] = mutual_info_classif(X, y)

# # Compute Spearman Correlation between each independent variable and the target
# spearman_coeffs = [pearsonr(X[col], y).correlation for col in X.columns]
# analysis_df['Pearson Correlation'] = spearman_coeffs

# Compute xicor Correlation between each independent variable and the target
XICOR_coeffs = [xicor(X[col], y)[0] for col in X.columns]
analysis_df['XICOR Correlation'] = XICOR_coeffs

analysis_df = analysis_df.sort_values(by=['VIF'])
analysis_df

,Feature,VIF,XICOR Correlation
4,snowfall_sum,1.007538,0.934503
3,Aspect,1.010808,0.037372
2,Slope,1.019092,0.052541
0,LC_Type1,1.295212,0.533846
1,avgLatitude,1.305279,-0.501033


In [ ]:
import pandas as pd
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
from scipy.stats import spearmanr, pearsonr
from sklearn.feature_selection import mutual_info_regression,mutual_info_classif
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

# Load your dataset
data = pd.read_csv('/content/gdrive/MyDrive/UND/Fires/FireBurntArea_dataset_0Days.csv',header=0)
labels = ['LC_Type1', 'avgLongitude', 'EVI', 'Albedo_BSA_nir', 'dayl', 'LST_Night_1km', 'Y', 'M', 'D', 'v_component_of_wind_10m', 'surface_pressure', 'snowmelt_sum'] + ['totalArea_km2']
data = data[labels]

# Separate the independent variables and the target variable
X = data.drop('totalArea_km2', axis=1)
y = data['totalArea_km2']

# Add a constant term for the intercept to the independent variables
X_const = add_constant(X)

# Initialize DataFrame to store the analysis results
analysis_df = pd.DataFrame()
analysis_df['Feature'] = X.columns

# Compute VIF
analysis_df['VIF'] = [variance_inflation_factor(X_const.values, i+1) for i in range(len(X.columns))]  # i+1 to skip constant
# analysis_df['Tolerance'] = 1 / analysis_df['VIF']

# # Compute Mutual Information
# analysis_df['Mutual Information'] = mutual_info_regression(X, y)

# # Compute Spearman Correlation between each independent variable and the target
# spearman_coeffs = [pearsonr(X[col], y).correlation for col in X.columns]
# analysis_df['Pearson Correlation'] = spearman_coeffs

# Compute xicor Correlation between each independent variable and the target
XICOR_coeffs = [xicor(X[col], y)[0] for col in X.columns]
analysis_df['XICOR Correlation'] = XICOR_coeffs

analysis_df = analysis_df.sort_values(by=['VIF'])
analysis_df

,Feature,VIF,XICOR Correlation
6,Y,1.062853,0.307656
9,v_component_of_wind_10m,1.063129,0.193392
8,D,1.160800,0.215576
11,snowmelt_sum,1.191340,0.297097
3,Albedo_BSA_nir,1.245317,0.146224
2,EVI,1.397128,0.189421
10,surface_pressure,1.450430,0.202626
0,LC_Type1,1.476907,0.253615
1,avgLongitude,1.620309,0.196415
5,LST_Night_1km,1.814234,0.197322


In [ ]:
import pandas as pd
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
from scipy.stats import spearmanr
from sklearn.feature_selection import mutual_info_regression,mutual_info_classif
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

# Load your dataset
data = pd.read_csv('/content/gdrive/MyDrive/UND/Fires/FireDuration_dataset_0Days.csv',header=0)
labels = ['LC_Type1', 'avgLongitude', 'avgLatitude', 'Slope', 'NDVI', 'Albedo_WSA_nir', 'Albedo_WSA_shortwave', 'prcp', 'srad', 'tmax', 'Y', 'M', 'D', 'v_component_of_wind_10m', 'runoff_sum', 'snow_depth', 'dewpoint_temperature_2m'] + ['Duration']
data = data[labels]

# Separate the independent variables and the target variable
X = data.drop('Duration', axis=1)
y = data['Duration']

# Add a constant term for the intercept to the independent variables
X_const = add_constant(X)

# Initialize DataFrame to store the analysis results
analysis_df = pd.DataFrame()
analysis_df['Feature'] = X.columns

# Compute VIF
analysis_df['VIF'] = [variance_inflation_factor(X_const.values, i+1) for i in range(len(X.columns))]  # i+1 to skip constant
# analysis_df['Tolerance'] = 1 / analysis_df['VIF']

# # Compute Mutual Information
# analysis_df['Mutual Information'] = mutual_info_regression(X, y)

# # Compute Spearman Correlation between each independent variable and the target
# spearman_coeffs = [pearsonr(X[col], y).correlation for col in X.columns]
# analysis_df['Pearson Correlation'] = spearman_coeffs

# Compute xicor Correlation between each independent variable and the target
XICOR_coeffs = [xicor(X[col], y)[0] for col in X.columns]
analysis_df['XICOR Correlation'] = XICOR_coeffs

analysis_df = analysis_df.sort_values(by=['VIF'])
analysis_df

,Feature,VIF,XICOR Correlation
13,v_component_of_wind_10m,1.135921,0.331472
10,Y,1.146913,0.513591
12,D,1.159955,0.407025
3,Slope,1.174827,0.315607
7,prcp,1.244163,0.500093
14,runoff_sum,1.678131,0.347814
11,M,1.732549,0.519042
1,avgLongitude,1.770354,0.332283
0,LC_Type1,1.801886,0.424537
4,NDVI,2.044509,0.367295
